In [13]:
#PromptTemplate

In [14]:
#字符模板 LLM chat
from langchain.prompts import PromptTemplate
import os

from dotenv import load_dotenv

load_dotenv()

prompt = PromptTemplate.from_template("你是一个{name}，帮我起一个具有{county}特色的男孩子名字")
prompt.format(name="算命大师",county="中国")


'你是一个算命大师，帮我起一个具有中国特色的男孩子名字'

ChatPromptTemplate

In [15]:
# 对话模板具有结构 chatmodels
from langchain.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一个起名大师，你的名字叫{name}"),
        ("human", "你好{name}，你感觉如何？"),
        ("ai", "我的状态还行"),
        ("human", "{user_input}"),
    ]
)
chat_template.format_messages(name="算命大师", user_input="你叫什么名字呢？")


[SystemMessage(content='你是一个起名大师，你的名字叫算命大师'),
 HumanMessage(content='你好算命大师，你感觉如何？'),
 AIMessage(content='我的状态还行'),
 HumanMessage(content='你叫什么名字呢？')]

In [16]:
# 包引入方式
from langchain.schema import SystemMessage
from langchain.schema import HumanMessage
from langchain.schema import AIMessage

#直接创建消息
sy = SystemMessage(
    content="你是一个起名大师",
    additional_kwargs={"大师姓名": "陈瞎子"}
)
hu = HumanMessage(
    content="请问大师叫什么"
)
ai = AIMessage(
    content="我叫程瞎子"
)
[sy, hu, ai]


[SystemMessage(content='你是一个起名大师', additional_kwargs={'大师姓名': '陈瞎子'}),
 HumanMessage(content='请问大师叫什么'),
 AIMessage(content='我叫程瞎子')]

ChatMessagePromptTemplate

In [17]:
from langchain.prompts import AIMessagePromptTemplate
from langchain.prompts import SystemMessagePromptTemplate
from langchain.prompts import HumanMessagePromptTemplate
from langchain.prompts import ChatMessagePromptTemplate

prompt = "愿{subjuct}与你同在!"

#ChatMessagePromptTemplate 可以自定义角色

chat_message_prompt = ChatMessagePromptTemplate.from_template(role="天行者", template=prompt)
chat_message_prompt.format(subjuct="神力")

ChatMessage(content='愿神力与你同在!', role='天行者')

In [18]:
# 自定义模板

In [36]:
#函数大师 根据函数名，查找代码，并给出中午的代码说明
from langchain.prompts import StringPromptTemplate

from dotenv import load_dotenv
import os

load_dotenv()

api_base = "https://oa.api2d.net/v1"
api_key = "fk226347-tr9eFgsLk3ZTfcZBPgfneDF1W0UD8zSw"
print(api_base, api_key)

#定义一个简单的函数

def hello_world(a):
    print("hello, world")
    return a

PROMPT = """\
你是一个非常有经验和天赋的程序员，现在给你如下函数名称，你会按照如下格式，输出这段代码的名称，源代码，中文解释。
函数名称：{function_name}
源代码:
{source_code}
代码解释:
"""

import inspect

def get_source_code(function_name):
    
    return inspect.getsource(function_name)

#自定义的模板Class
class CustmPrompt(StringPromptTemplate):
    
    def format(self, **kwargs) -> str:
        
        #获得源代码
        source_code = get_source_code(kwargs['function_name'])

        #生成提示词模板
        prompt = PROMPT.format(
            function_name=kwargs['function_name'].__name__, source_code=source_code
        )
        return prompt
    

a = CustmPrompt(input_variables=["function_name"])
pm = a.format(function_name=hello_world)

print(pm)

#hello world
# from langchain.llms import OpenAiChat  
# from langchain.chat_models import ChatOpenAI
from langchain_openai import ChatOpenAI

llm = ChatOpenAI (
    model="gpt-3.5-turbo",
    temperature=0,
    max_tokens=1000,
    openai_api_base=api_base,
    openai_api_key=api_key,  # this is also the default, it can be omitted
)
# msg = llm.predict(pm) # LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 0.2.0. Use invoke instead
msg = llm.invoke(pm)  # 0.1.7 以后

print(msg.content)  #0.1.7以后用法

https://oa.api2d.net/v1 fk226347-tr9eFgsLk3ZTfcZBPgfneDF1W0UD8zSw
你是一个非常有经验和天赋的程序员，现在给你如下函数名称，你会按照如下格式，输出这段代码的名称，源代码，中文解释。
函数名称：hello_world
源代码:
def hello_world(a):
    print("hello, world")
    return a

代码解释:

函数名称：hello_world

源代码:
```python
def hello_world(a):
    print("hello, world")
    return a
```

中文解释:
这是一个名为hello_world的函数，它接受一个参数a，并打印出"hello, world"，然后返回参数a。
